In [2]:
import sys
sys.path.append("../../../MVCNN")
import torch
from tools.ImgDataset import MultiviewImgDataset
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.nn import functional as F

In [3]:
pruned_models = {
    0.0: torch.jit.load("/Users/mahadevsunilkumar/Desktop/DDNN-Working/src/pruning/taylor_series/incremental-pruning/mvcnn-pruned/models/mvcnn_pruned_0.00.pth", map_location=torch.device('mps')),
    0.1: torch.jit.load("/Users/mahadevsunilkumar/Desktop/DDNN-Working/src/pruning/taylor_series/incremental-pruning/mvcnn-pruned/models/mvcnn_pruned_0.10.pth", map_location=torch.device('mps')),
    0.3: torch.jit.load("/Users/mahadevsunilkumar/Desktop/DDNN-Working/src/pruning/taylor_series/incremental-pruning/mvcnn-pruned/models/mvcnn_pruned_0.30.pth", map_location=torch.device('mps')),
    0.5: torch.jit.load("/Users/mahadevsunilkumar/Desktop/DDNN-Working/src/pruning/taylor_series/incremental-pruning/mvcnn-pruned/models/mvcnn_pruned_0.50.pth", map_location=torch.device('mps')),
    0.7: torch.jit.load("/Users/mahadevsunilkumar/Desktop/DDNN-Working/src/pruning/taylor_series/incremental-pruning/mvcnn-pruned/models/mvcnn_pruned_0.70.pth", map_location=torch.device('mps')),
    0.9: torch.jit.load("/Users/mahadevsunilkumar/Desktop/DDNN-Working/src/pruning/taylor_series/incremental-pruning/mvcnn-pruned/models/mvcnn_pruned_0.90.pth", map_location=torch.device('mps')),
}

In [4]:
test_path = "../../../MVCNN/ModelNet40-12View/*/test"
train_path = "../../../MVCNN/ModelNet40-12View/*/train"
device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
num_views=12

In [5]:
dataset = MultiviewImgDataset(
    root_dir=train_path,
    scale_aug=False,
    num_models=0,
    num_views=12,
)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=4, pin_memory=True)
print(f"Samples in dataset: {len(dataset)}")

Samples in dataset: 7589


In [6]:
dataset[0][-1]

['../../../MVCNN/ModelNet40-12View/vase/train/vase_0261//vase_0261/vase_0261.0.png',
 '../../../MVCNN/ModelNet40-12View/vase/train/vase_0261//vase_0261/vase_0261.1.png',
 '../../../MVCNN/ModelNet40-12View/vase/train/vase_0261//vase_0261/vase_0261.10.png',
 '../../../MVCNN/ModelNet40-12View/vase/train/vase_0261//vase_0261/vase_0261.11.png',
 '../../../MVCNN/ModelNet40-12View/vase/train/vase_0261//vase_0261/vase_0261.2.png',
 '../../../MVCNN/ModelNet40-12View/vase/train/vase_0261//vase_0261/vase_0261.3.png',
 '../../../MVCNN/ModelNet40-12View/vase/train/vase_0261//vase_0261/vase_0261.4.png',
 '../../../MVCNN/ModelNet40-12View/vase/train/vase_0261//vase_0261/vase_0261.5.png',
 '../../../MVCNN/ModelNet40-12View/vase/train/vase_0261//vase_0261/vase_0261.6.png',
 '../../../MVCNN/ModelNet40-12View/vase/train/vase_0261//vase_0261/vase_0261.7.png',
 '../../../MVCNN/ModelNet40-12View/vase/train/vase_0261//vase_0261/vase_0261.8.png',
 '../../../MVCNN/ModelNet40-12View/vase/train/vase_0261//vase_0

In [14]:
next(iter(dataloader))[1][0][11].shape

torch.Size([3, 224, 224])

In [ ]:
def get_accuracy(model):
    all_correct = 0
    all_samples = 0
    
    wrong_class = np.zeros(33, dtype=int)
    samples_class = np.zeros(33, dtype=int)
    
    model.eval()
    model = model.to(device)
    with torch.no_grad():
        pbar = tqdm(dataloader, total=len(dataloader), desc="Testing")
        for _, data in enumerate(pbar):
            labels, images = data[0], data[1]
            labels = labels.to(device)
            images = images.to(device)       
            N, V, C, H, W = images.size()
            
            x = images.view(-1, C, H, W)
            y = model.net_1(x)
            # print(f"after net_1: {y.shape}")
            y = y.view((int(x.shape[0]/num_views),num_views,y.shape[-3],y.shape[-2],y.shape[-1]))
            # print(f"after reshape: {y.shape}")
            y = torch.max(y, dim=1)[0]
            # print(f"after max pooling: {y.shape}")
            y = y.view(y.shape[0], -1)
            # print(f"after reshape: {y.shape}")
            preds = model.net_2(y)
            # print(f"after net_2: {preds.shape}")
            preds = preds.argmax(dim=1)
            batch_correct = (preds == labels).sum().item()
            all_correct += batch_correct
            all_samples += N    
            for i in range(N):
                if preds[i] != labels[i]:
                    wrong_class[labels[i].cpu().item()] += 1
                samples_class[labels[i].cpu().item()] += 1

        overall_acc = all_correct / all_samples
        per_class_acc = (samples_class - wrong_class) / np.maximum(samples_class, 1)
        mean_class = per_class_acc[samples_class > 0].mean()

        print(f"Overall accuracy: {overall_acc:.4f}")
        print(f"Mean class accuracy: {mean_class:.4f}")

In [ ]:
for key in pruned_models.keys():
    model = pruned_models[key]
    print(f"Testing model with pruning rate {key}")
    get_accuracy(model)
    print("-" * 50)
    print()

Testing model with pruning rate 0.0


Testing:   0%|          | 0/7589 [00:00<?, ?it/s]

after net_1: torch.Size([12, 512, 7, 7])
after reshape: torch.Size([1, 12, 512, 7, 7])
after max pooling: torch.Size([1, 512, 7, 7])
after reshape: torch.Size([1, 25088])
after net_2: torch.Size([1, 33])


Testing:   0%|          | 0/7589 [00:09<?, ?it/s]
/var/folders/_g/s3cg441n5772k0djn_3x0frc0000gp/T/ipykernel_16188/3234337798.py:41: RuntimeWarning: Mean of empty slice.
  mean_class = per_class_acc[samples_class > 0].mean()
/opt/homebrew/Caskroom/miniconda/base/envs/ML/lib/python3.13/site-packages/numpy/_core/_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Overall accuracy: 1.0000
Mean class accuracy: nan
--------------------------------------------------

Testing model with pruning rate 0.1


...

Testing:   0%|          | 0/7589 [00:00<?, ?it/s]

after net_1: torch.Size([12, 512, 7, 7])
after reshape: torch.Size([1, 12, 512, 7, 7])
after max pooling: torch.Size([1, 512, 7, 7])
after reshape: torch.Size([1, 25088])
after net_2: torch.Size([1, 33])


Testing:   0%|          | 0/7589 [00:09<?, ?it/s]


Overall accuracy: 1.0000
Mean class accuracy: nan
--------------------------------------------------

Testing model with pruning rate 0.5


Testing:   0%|          | 0/7589 [00:00<?, ?it/s]

In [ ]:
random_img = torch.randn(1, 12, 3, 224, 224).to(device)
N, V, C, H, W = random_img.size() 

In [ ]:
out = pruned_models[0.5].net_1(random_img.view(-1, C, H, W))

In [ ]:
y = F.adaptive_avg_pool2d(out, (7, 7)).view(12, -1)

In [ ]:
y.shape

In [ ]:
y = torch.max(y, 0)[0].unsqueeze(0)
y.shape

In [ ]:
model.net_2(y)

In [ ]:
torch.zeros((1, 512, 12), device=device)

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], device='mps:0')